In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
%matplotlib inline

In [ ]:
folder= tb.latest_data('')
#folder= tb.latest_data('122849'); print 'OLDER FOLDER !!!'
#folder=r'D:\measuring\data\20140918\122229_ElectronRabi_Pippin_rabi'
#ssro_folder = tb.latest_data('155255_AdwinSSRO_SSROCalibration_Pippin_SIL3') #tb.data_from_time('20140708183041')
ssro_folder = tb.latest_data('SSRO')
a=sequence.SequenceAnalysis(folder)
#test =a.get_sweep_pts()
#print a.get_readout_results(name='ssro')
a.get_electron_ROC(ssro_calib_folder=ssro_folder)
ax = a.plot_result_vs_sweepparam(ret='ax', name='ssro')
ax.set_ylim(.0,1.1)
ax.set_xlim(0,50)
print a.sweep_pts
#y=a.p0
#np.savetxt('rabi_2.txt', (x,y))

In [ ]:
stfolder_1= tb.latest_data('110106')
folder_2= tb.latest_data('105928'); print 'ORLDER FOLDER !!!'
#folder=r'D:\measuring\data\20140707\142326_DD_CORPSE_Sam_SIL5'
ssro_folder = tb.latest_data('SSROCalibration') #tb.data_from_time('20140708183041')
a_1=sequence.SequenceAnalysis(folder_1)
a_1.get_sweep_pts()
a_1.get_readout_results(name='ssro')
a_1.get_electron_ROC(ssro_calib_folder=ssro_folder)
a_2=sequence.SequenceAnalysis(folder_2)
a_2.get_sweep_pts()
a_2.get_readout_results(name='ssro')
a_2.get_electron_ROC(ssro_calib_folder=ssro_folder)
x_1= a_1.sweep_pts
y_1 = a_1.p0
x_2= a_2.sweep_pts
y_2= a_2.p0
fig_0, ax_0 = plt.subplots(1,1, figsize=(4.5,4))
#ax = plt.plot(x_1, y_1,'bo', x_2, y_2,'ro')
ax_0.errorbar(x_1, y_1, yerr = a_1.u_p0, fmt='bo')
ax_0.errorbar(x_2, y_2, yerr = a_2.u_p0, fmt='ro' )
ax_0.set

In [ ]:
execfile(r'D:\measuring\analysis\scripts\espin\electron_rabi_analysis.py')

In [ ]:
import os, sys
import numpy as np
import h5py
import logging

from matplotlib import pyplot as plt

from analysis.lib import fitting
from analysis.lib.m2.ssro import  sequence, mbi #sequence_ssro,
from analysis.lib.tools import toolbox
from analysis.lib.fitting import fit, rabi
reload(rabi)
reload(sequence)

from analysis.lib.tools import plot

guess_frq = 1./0.12

guess_amp = 0.4
guess_of = 0.8
# guess_slope = 0.
guess_phi = 0.
guess_k = 0.

mbi_analysis = False

o = fit.Parameter(guess_of, 'o')
f = fit.Parameter(guess_frq, 'f')
A = fit.Parameter(guess_amp, 'A')
phi = fit.Parameter(guess_phi, 'phi')
k = fit.Parameter(guess_k, 'k')
p0 = [f, A, phi, o, k]
fitfunc_str = ''

folder= tb.latest_data('Rabi')
#folder='D:\measuring\data\20141015\163957_ElectronRabi_Pippin_rabi'


a = sequence.SequenceAnalysis(folder)
print a
a.get_sweep_pts()
a.get_readout_results('ssro')
a.get_electron_ROC()
ax = a.plot_result_vs_sweepparam(ret='ax')

x = a.sweep_pts
y = a.p0


# fit_result = fit.fit1d(x, y, rabi.fit_rabi_multiple_detunings,
#         guess_amp, guess_yof, guess_frq, guess_tau, (0, 0), (-2.193e-3, 0), (2.193e-3, 0), fixed=[],
#         do_print=True, ret=True)

fitfunc_str = 'o - A + A*e^(-(kx)**2)*cos(2pi (fx-phi))'

def fitfunc(x):
    return (o()-A()) + A() * np.exp(-(k()*x)**2) * np.cos(2*np.pi*(f()*x - phi()))

fit_result = fit.fit1d(x,y, None, p0=p0, fitfunc=fitfunc, fixed=[2,4],
        do_print=True, ret=True)
plot.plot_fit1d(fit_result, np.linspace(0,x[-1],201), ax=ax,
        plot_data=False)
ax.set_xlim(0.0,0.1)
print "pi pulse = {:.3f} ".format(1/f()/2.) + a.sweep_name

# ax.set_title(a.timestamp+'\n'+a.measurementstring)
plt.savefig(os.path.join(folder, 'electronrabi_analysis_fit.png'))


In [ ]:
from analysis.scripts.espin import simul_Rabi
reload(simul_Rabi)
simul_Rabi.sweep_pulse_ampl(pulse_duration =2000e-9, Rabi_freq_max =2e6, detuning = 0.e6)
#simul_Rabi.sweep_pulse_duration(pulse_duration_max = 4e-6, Rabi_freq = 0.26e6,detuning = 0e6)

In [ ]:
execfile(r'D:\measuring\analysis\scripts\espin\dark_esr_analysis.py')

In [ ]:
from analysis.scripts.espin import analysis_espin_calibration as aec
reload(aec)
folder = toolbox.latest_data('DarkESR')
test=aec.analyse_dark_esr()

In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.scripts.bell import calibration_tools
reload(calibration_tools)
folder=tb.latest_data('Pi_Calibration')
fig, ax = plt.subplots(1,1, figsize=(4.5,4))
ymin=-0.0
ymax = 0.1
fit=calibration_tools.fit_parabolic(folder, x0_guess=0.92,a_guess=-2,of_guess=0.02, ax=ax, info_xy=(0.88,ymin-(ymax-ymin)*0.35))
#fit=calibration_tools.fit_parabolic(folder, x0_guess=1e-9,a_guess=-2e14,of_guess=0.02, ax=ax, info_xy=(-5e-9,-0.02))
fit_result=calibration_tools.plot_result(folder, ax=ax, ret=True)
ax.set_ylim(ymin,ymax)
print 1-fit['params'][0]
print fit['params'][2]
#print 'Fitted minimum at ', fit['params'][2]

In [ ]:
calibration_tools.fit_parabolic?

In [ ]:
execfile(r'D:\measuring\analysis\scripts\espin\calibration_pi2_CORPSE.py')

In [ ]:
print sqrt(sin(pi/4.))


In [ ]:
execfile(r'D:\measuring\analysis\scripts\espin\electron_ramsey_analysis.py')

In [ ]:
ramsey_fft=np.fft.fft(a.p0)
N_pts_fft = len(a.sweep_pts)/2
ramsey_fft_freq = np.linspace(0,0.5/(a.sweep_pts[-1]/float(N_pts_fft)),N_pts_fft)*1e3
#ramsey_fft_freq = np.fft.fftfreq(a.sweep_pts.shape[-1])
print N_pts_fft
fig,ax=plt.subplots(1,1)
#plt.plot(ramsey_fft_freq[N_pts_fft:], ramsey_fft.real[N_pts_fft:])
plt.plot(ramsey_fft_freq, ramsey_fft.real[N_pts_fft:])
#ax.set_ylim([0,4])
print 1/(a.sweep_pts[-1]/len(a.sweep_pts))*1e3


In [ ]:
4/8.

In [ ]:
from analysis.scripts.espin import simpledecoupling_analysis as s
reload(s)
result=s.electron_T2_anal( measurement_name=['ssro'], ylim=(0.8,1))
print '{:.3f} +/- {:.3f}'.format(result[0]['params'][1]+ result[0]['params'][2],result[0]['error'][1]+ result[0]['error'][2])

In [ ]:
a=arange(10)*3
a=a[0:2]
print a

In [ ]:
from analysis.scripts.bell import calibration_tools
reload(calibration_tools)
folder=tb.latest_data('135925')
fit=calibration_tools.fit_parabolic(folder, x0_guess=-30,a_guess=2.)
fit_result=calibration_tools.plot_result(folder, ret=True)
print 'Fitted maximum at ', fit['params'][2], 'us'
print ' {:.3f} +/- {:.3f}'.format(1-fit['params'][0]+fit['params'][1], np.sqrt(fit['error'][0]**2+fit['error'][1]**2))

In [ ]:
execfile(r'D:\measuring\analysis\scripts\espin\simple_esr_fit.py')

In [ ]:
from analysis.lib.fitting import fit, ramsey
from analysis.lib.tools import plot
reload(ramsey)

ssro_calib_folder = toolbox.latest_data(contains='130113_AdwinSSRO_SSROCalibration') 
folder= tb.latest_data('')
a = sequence.SequenceAnalysis(folder)

a.get_sweep_pts()
a.get_readout_results('ssro')
a.get_electron_ROC(ssro_calib_folder=ssro_calib_folder)
x= a.sweep_pts
y=a.p0

tau = 4000      # ns
amplitude = -0.03       # amplitude
offset = 0.5        # offset
detuning = 3.0e-3 # in GHz
hf_N = 2.19e-3  # 14N hyperfine splitting in GHz
#hf_C = 0.858e-3   # 13C hyperfine splitting in GHz as **kw



fit_result = fit.fit1d(x,y,ramsey.fit_ramsey_14N_fixed_13C_opt,tau, amplitude, offset,detuning,hf_N, ret=True)
params_0, fitfunc_0, fitfunc_str= ramsey.fit_ramsey_14N_fixed_13C_opt(tau, amplitude, offset,detuning,hf_N)
test=fitfunc_0(x)
plot.plot_fit1d(fit_result,x,info_xy=(1000,-0.5))
ax=gca()
plt.plot(x, fitfunc_0(x), 'r--')
ax.set_ylim(0,1.2)
ax.set_title(a.timestamp)
ax.set_ylabel(r'$F(|0\rangle)$')
ax.set_xlabel(a.sweep_name)

file = open("ramsey_121606.txt", "w")

file.write("data")

file.write(x)

file.close()


In [ ]:
savetxt('ramsey_130414_x.txt', x)

In [ ]:
from analysis.scripts.espin import analysis_espin_calibration as aec
reload(aec)
folder = toolbox.latest_data('')
print folder
test = aec.analyse_Rabi(guess_frq =1/0.03 )

In [ ]:
from analysis.scripts.bell import calibration_tools
reload(calibration_tools)
folder=tb.latest_data('Bell')
a,ax,x,y=calibration_tools.plot_result(folder, ret=True)
fit_result =calibration_tools.fit_gaussian(folder, ax=ax, x0_guess = 75,c_guess = 20, a_guess= -0.5)
#ax.set_ylim([0.5,1.05])
print 'maximum at', fit_result['params'][1]*1e6, ' us'

##### Calibrate pi/4:

In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.scripts.bell import calibration_tools
reload(calibration_tools)
folder=tb.latest_data('')
fig, ax = plt.subplots(1,1, figsize=(4.5,4))
fit=calibration_tools.fit_linear(folder, a_guess = 1., b_guess=0., ax=ax)
fit_result=calibration_tools.plot_result(folder, ax=ax, ret=True)
ax.set_ylim(0.5,1)
print 'pi/4 (1/2.*(1+cos(pi/4.) at:',(1/2.*(1+cos(pi/4.))-fit['params_dict']['b'])/fit['params_dict']['a']
print 'error:',sqrt((1/2.*(1+cos(pi/4.))-fit['params_dict']['b'])**2/fit['params_dict']['a']**2*fit['error_dict']['a']**2 + 1./fit['params_dict']['a']**2*fit['error_dict']['b']**2 )
